In [ ]:
import pandas as pd
import requests
from keys import KEY_TWO


In [ ]:
# API Call to get list of LEGO themes and convert to dataframe
themes = requests.get(f'https://brickset.com/api/v3.asmx/getThemes?apiKey={KEY_TWO}') 
data = themes.json()
df = pd.json_normalize(data, 'themes')


In [ ]:
# Drop themes older than 1999, themes with less than 50 sets, sets that aren't currently in production, minifig theme, and miscellaneous theme
mask = df[(df['yearFrom'] < 1999) | (df['setCount'] < 50) | (df['yearTo'] < 2022) |
         (df['theme'] == 'Collectable Minifigures') | (df['theme'] == 'Miscellaneous')].index
df.drop(mask, inplace=True)
df.shape

In [ ]:
# Generate sample theme list to use in 2nd API call. Convert list to string for API parameters.
theme_list = []
for item in df['theme'].sample(3):
    theme_list.append(item)
param_string = ", ".join(theme_list)
param_string

In [ ]:
# 2nd API call to get a full set list for themes in the theme list generated by first API call. Convert to a data frame
parameters = {'theme' : f'{param_string}', 'pageSize' : 2500}
set_list = requests.get(f"https://brickset.com/api/v3.asmx/getSets?apiKey={KEY_TWO}&userHash=&params={parameters}")
set_data = set_list.json()
set_df = pd.json_normalize(set_data,'sets')
set_df.shape


In [ ]:
# List of columns to drop from Dataframe. Columns not relevant to analysis.
drop_list = [
            'numberVariant',
            'released',
            'category',
            'bricksetURL',
            'reviewCount',
            'packagingType',
            'availability',
            'instructionsCount',
            'additionalImageCount',
            'lastUpdated',
            'image.thumbnailURL',
            'image.imageURL',
            'collections.ownedBy',
            'collections.wantedBy',
            'dimensions.height',
            'dimensions.width',
            'dimensions.depth',
            'LEGOCom.US.retailPrice',
            'LEGOCom.US.dateFirstAvailable',
            'LEGOCom.US.dateLastAvailable',
            'LEGOCom.UK.retailPrice',
            'LEGOCom.UK.dateFirstAvailable',
            'LEGOCom.UK.dateLastAvailable',
            'LEGOCom.CA.retailPrice',
            'LEGOCom.CA.dateFirstAvailable',
            'LEGOCom.CA.dateLastAvailable',
            'dimensions.weight',
            'barcode.EAN',
            'barcode.UPC',
            'minifigs',
            'LEGOCom.DE.retailPrice',
            'LEGOCom.DE.dateFirstAvailable',
            'LEGOCom.DE.dateLastAvailable'
            ]
set_df.drop(columns=drop_list, inplace=True)
set_df

In [ ]:
# Drop rows where there is no rating for the set.
mask_two = set_df[set_df['rating'] == 0].index
set_df.drop(mask_two, inplace=True)
set_df.shape

In [ ]:
# Drop any rows if they have a NaN value in the pieces column
pieces_null = set_df['pieces'].isnull().values.any()
print(pieces_null)
if pieces_null == True:
    set_df.dropna(subset=['pieces'], inplace=True)
set_df.shape

In [ ]:
#Convert pieces, minimum age range, and maximum age range from floats to ints.
set_df['pieces'] = set_df['pieces'].astype(pd.Int64Dtype())
set_df['ageRange.min'] = set_df['ageRange.min'].astype(pd.Int64Dtype())
set_df['ageRange.max'] = set_df['ageRange.max'].astype(pd.Int64Dtype())

In [ ]:
rename_dict = {
               'setID' : 'Set ID',
               'number' : 'Set Number',
               'name' : 'Set Name',
               'year' : 'Release Year',
               'theme' : 'Theme',
               'themeGroup' : 'Theme Group',
               'subtheme' : 'Subtheme',
               'pieces' : 'Number of Pieces',
               'rating' : 'Brickset Rating',
               'ageRange.min' : 'Min Age Range',
               'ageRange.max' : 'Max Age Range',
                }
                

set_df.rename(columns=rename_dict, inplace=True)
set_df

In [ ]:
set_df = set_df.sample(100)

In [ ]:
# Save data to CSV for visualization in Tableau
set_df.to_csv('./set_list.csv')